In [4]:
from crystalbuilder import *
import crystalbuilder.geometry as geo
import matplotlib.pyplot as plt
import tidy3d as td
import crystalbuilder.convert as cv
import numpy as np
import vedo

Error: Lumpy and/or the Lumerical API were not found.


Here we will create a simulation of the core-shell diamond crystal. This is a rod-connected diamond with a cubic conventional cell. We'll set it up in the usual manner, but we can use an actual distance in nanometers for our lattice constant. `a_mag` is this constant in microns (Tidy3D default units)

In [5]:
lattice_constant = 700 #nm

a1 = [1, 0, 0]
a2 = [0, 1 ,0]
a3 = [0, 0, 1]

a_mag = lattice_constant/1000 #convert to microns
geo_lattice = lattice.Lattice(a1, a2, a3, magnitude = [a_mag, a_mag, a_mag])

We'll load our points from Bilbao (or a local copy if already done before). We don't scale any of this using the lattice constant.  

In [6]:
diamond = bilbao.SpaceGroup(227)
pt = 1/8
points = diamond.calculate_points(point_list=[(pt, pt, pt)]) #This is the 8a wyckoff position

The IUCr tabulated general positions are in units of the lattice constant. After we connect the nearest neighbors, we convert to our micron units and scale the structure. This is done by passing `a_mag` to `NearestNeighbors()`. 

In [7]:
radius = .05 #microns

crystal_core = geo.NearestNeighbors(points, radius=radius, neighborhood_range=.45, a_mag=a_mag)

For our core-shell structure, we will just create a slightly larger (by `shell_radius`) set of connected rods. 

In [23]:
shell_radius = .0125 #microns

crystal_shell = geo.NearestNeighbors(points, radius=(radius+shell_radius), neighborhood_range=.45, a_mag=a_mag)

The geometry is just a combination of these two lattices, **making sure to include the core *after* the shell**. In Tidy3D, items later in the list will override earlier ones.

In [24]:
geometry = [crystal_shell, crystal_core]

In [25]:
a1_reps = 6
a2_reps = 6
a3_reps = 4
crystal = geo_lattice.tile_geogeometry(geometry, a1_reps, a2_reps, a3_reps )
print(len(crystal[0]))
print(len(crystal[1]))

2304
2304


In [26]:
scene = viewer.visualize(crystal[1], c='white', res=5)
viewer.add_to_visualizer(crystal[0], scene, c='red', alpha=.5, res=5)


# scene.show().close()

In [27]:
shell = cv.geo_to_tidy3d(crystal[0], 3.4, name="Shell", material_name="HPCVD-Si")
core = cv.geo_to_tidy3d(crystal[1], 1.75, name="Core", material_name = "FEBID-Pt")

In [29]:
# create source
lda0 = 1.25  # wavelength of interest (length scales are micrometers in Tidy3D)
src_wavelengths = [.7, 1.6]

src_freqs = np.sort(td.C_0 / np.asarray(src_wavelengths))
freq0 = np.mean(src_freqs)  # frequency of interest
print(src_freqs)


[1.87370286e+14 4.28274940e+14]


AttributeError: 'numpy.ndarray' object has no attribute 'json'

In [34]:

def make_sources(frequencies, size, theta_range=[0, 1.2], phi_range = [0, 2*np.pi], number_of_angles = 5, **kwargs):
    
    center = kwargs.get("center", (0,0,2))
    direction = kwargs.get("direction", '-')
    phi = np.arange(phi_range[0], phi_range[1], .275)
    theta = np.linspace(theta_range[0], theta_range[1], number_of_angles)
    num_mons = (len(theta)-1) * len(phi) +1
    source_list = []
    time_profile = td.GaussianPulse.from_frequency_range(fmin=frequencies[0], fmax=frequencies[1], amplitude = 1/num_mons )
    for aang in theta:
        if aang == 0:
            scaled_time_profile = td.GaussianPulse.from_frequency_range(fmin=frequencies[0], fmax=frequencies[1])
            source = td.GaussianBeam(
                size = size,
                center = center,
                source_time = scaled_time_profile,
                direction=direction,
                waist_radius=2.5,
                angle_theta=aang,
                angle_phi=0,
                name="Normal Incidence"
            )
            source_list.append(source)
            pass
        else:
            for rang in phi:
                source = td.GaussianBeam(
                    size = size,
                    center = center,
                    source_time = time_profile,
                    direction=direction,
                    waist_radius=2.5,
                    angle_theta=aang,
                    angle_phi=rang,
                    num_freqs=15
                )
                source_list.append(source)
    
    return source_list

sources = make_sources(src_freqs, theta_range=[0, 1], size=[4,4,0], number_of_angles = 15, center=(-.1, 0, 2))


nmonitor_freqs = 25 #Number of frequencies to sample at.

monitor_freqs = np.linspace(src_freqs[0], src_freqs[1] ,nmonitor_freqs)

farfield_monitor = td.FieldProjectionAngleMonitor(center=[0,0,2.25], 
                                                  size=[td.inf,td.inf,0],
                                                  freqs=(monitor_freqs),
                                                  name='farfield_monitor',
                                                  phi = np.linspace(0, 2*np.pi, 100),
                                                  theta=np.linspace(0,1.2, 70))

fieldmon_size = [8,8,0]

flux_t_monitor = td.FieldMonitor(center=[0,0,-2], 
                                size=fieldmon_size,
                                freqs=(monitor_freqs),
                                name='trans_monitor'  
)

realflux_r = td.FluxMonitor(center=[0,0,2.25], 
                                size=fieldmon_size,
                                freqs=(monitor_freqs),
                                name='refl_fluxmonitor')

flux_r_monitor = td.FieldMonitor(center=[0,0,2.25], 
                                size=fieldmon_size,
                                freqs=(monitor_freqs),
                                name='refl_monitor')

YZ_monitor = td.FieldMonitor(center=[0,0,0], 
                                size=[0,3,3],
                                freqs=(monitor_freqs),
                                name='YZ_monitor')




monitor_list = [farfield_monitor, flux_t_monitor, flux_r_monitor, YZ_monitor,  realflux_r]

simulation_attributes = {
    "sources" :{
        "frequency_min": src_freqs[0],
        "frequency_max": src_freqs[1],
        "frequency_center": freq0,
        "number": len(sources),
    }
    
}



sim = td.Simulation(
    attrs=simulation_attributes,
    size=(13, 13, 6),  # simulation domain size
    center = (-0.1,0,0),
    grid_spec=td.GridSpec.auto(
        min_steps_per_wvl=20
    ),  # automatic nonuniform FDTD grid with 25 grids per wavelength in the material
    structures=[shell, core],
    sources=sources,
    monitors=monitor_list,
    run_time=3e-13,  # physical simulation time in second
) # type: ignore
sim.plot_3d()





In [35]:
from tidy3d import web

task_id = web.upload(sim, task_name=f"{a1_reps}x{a2_reps}x{a3_reps}-CoreShell-NormalizationAgain", folder_name="Multi-Gaussian Reflection", verbose=True)

09:15:53 Central Daylight Time Created task '6x6x4-CoreShell-NormalizationAgain'
                               with task_id                                     
                               'fdve-6683237c-137c-4c7f-b03a-ce97edd9b935' and  
                               task_type 'FDTD'.

                               View task using web UI at                        
                               ]8;id=155462;https://tidy3d.simulation.cloud/workbench?taskId=fdve-6683237c-137c-4c7f-b03a-ce97edd9b935\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=755228;https://tidy3d.simulation.cloud/workbench?taskId=fdve-6683237c-137c-4c7f-b03a-ce97edd9b935\taskId]8;;\
                               ]8;id=155462;https://tidy3d.simulation.cloud/workbench?taskId=fdve-6683237c-137c-4c7f-b03a-ce97edd9b935\=]8;;\]8;id=142558;https://tidy3d.simulation.cloud/workbench?taskId=fdve-6683237c-137c-4c7f-b03a-ce97edd9b935\fdve]8;;\]8;id=155462;https://tidy3d.simulation.cloud/workbench?taskId=fdve-6683237c-137c-4c7f-b03a-ce97edd9b935\-6683237c-137c-4c7f-b03a-ce97edd9b935']8;;\.

                               Task folder: ]8;id=414300;https://tidy3d.simulation.cloud/folders/folder-066bea7d-969e-4af3-be54-8e64815456f0\'Multi-Gaussian Reflection']8;;\.

Output()

09:17:02 Central Daylight Time Maximum FlexCredit cost: 0.162. Minimum cost     
                               depends on task execution details. Use           
                               'web.real_cost(task_id)' to get the billed       
                               FlexCredit cost after a simulation run.